# 필요 라이브러리

## API 공식 문서 : https://developers.google.com/youtube/v3/getting-started
## 라이브러리 공식 저장소 : https://github.com/googleapis/google-api-python-client

pip install google-api-python-client

# 메소드 활용법

build = 인증 객체 ('api서비스이름','api 버전', api키)  / 마지막에 객체.close() 해주는게 좋음(소캣 열려있는것 방지) <br>
인증객체.collections() => 리소스 집합 <br>
인증객체.collections().method() => 리소스 메소드   / HttpRequest 객체를 반환 (except HttpError 로 애러 확인 가능))


## 부분 리소스
### 할당량 관리를 위해 사용되는 속성

part : 반환되어야 하는 속성 그룹(반드시 써야됨) <br>
fields : 리소스 내에 특정 속성만 반환하기 위해서만 api 응답 필터링 <br>

활용 예시

In [ ]:
from googleapiclient.discovery import build
import json

#동일 경로에 APIkey.json 파일 안에 APIkey 저장

with open('APIkey.json') as json_file:
    json_data = json.load(json_file)
    youtubeApiKey = json_data["APIkey"]

youtube = build('youtube','v3',developerKey=youtubeApiKey)
channelId='UCUbOogiD-4PKDqaJfSOTC0g'


In [ ]:
statdata = youtube.channels().list(part='statistics', id=channelId).execute()
statdata

In [ ]:
#json 형식으로 깔끔하게 보기

print(json.dumps(statdata, sort_keys=True, indent=4))

# 각 리소스 별 메소드 정리 <br><br>

## 지원 메소드 공식 문서 : https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.html

테스터훈 채널 id = UCUbOogiD-4PKDqaJfSOTC0g <br>
감스트 채널 id = UCbFzvzDu17eDZ3RIeaLRswQ <br>
내 채널 id = UC-mf4T0qrxiBjGzY8eUTMsQ

In [ ]:
# Activites 특정 채널이나 사용자가 YouTube에서 실행한 작업의 정보가 포함되어 있습니다. 활동 피드로 보고되는 작업에는 동영상 평가, 동영상 공유, 동영상을 즐겨찾기에 추가, 동영상에 댓글 달기, 동영상 업로드 등이 있습니다.

# 영상 업로드 기록만 나온다. 영상 제목과 설명 뽑기에 사용할 수 있을듯.


activites_result = []
newpageToken = ''

for i in range(1,7):    
    activites = youtube.activities().list(part=' snippet, contentDetails', channelId='UCbFzvzDu17eDZ3RIeaLRswQ' , fields='pageInfo, nextPageToken, items(contentDetails, snippet(title, description, type))', pageToken=newpageToken).execute()
    newpageToken = activites['nextPageToken']
    activites_result.append(activites)
    print(newpageToken)
    print('test %d clear' %i)

activites_result



In [ ]:
# channel YouTube 채널에 대한 정보가 포함되어 있습니다
# part : id, snippet, brandingSettings, contentDetails, invideoPromotion, statistics, topicDetails  -- invideoPromotion은 더이상 지원안하는듯 

#채널명, 시작일, 조회수, 구독자수, 업로드비디오수, 토픽(카테고리), 채널설명, 채널키워드, 배너 이미지  수집 가능

channels = youtube.channels().list(part='snippet, brandingSettings, contentDetails,  statistics, topicDetails', id='UCbFzvzDu17eDZ3RIeaLRswQ', fields='items(brandingSettings, contentDetails, statistics, topicDetails, snippet(title, publishedAt, country))').execute()
channels_result = []
channels_result.append(channels)
channels_result


In [ ]:
#channelSections  information about a set of videos that a channel has chosen to feature.

#해당채널의 재생목록 (채널 첫 페이지에 있는 것) 확인 가능 + 유튜버에 따라 본인이 설정한 추천 유튜버 목록도 수집 가능


channelSections = youtube.channelSections().list(part='contentDetails, localizations, snippet, targeting', channelId='UCbFzvzDu17eDZ3RIeaLRswQ').execute()
channelSections_result = []
channelSections_result.append(channelSections)
channelSections_result


In [ ]:
#comment   information about a single YouTube comment
#댓글 고유 id를 얻어야됨 -> commentThreads 메소드 먼저 사용해야함.`
#commentThreads 랑 다른 정보 없음 사용할 필요 없음

# comments = youtube.comments().list(part='id, snippet', id='UCJ1pToolWeekW9VnkqghB-w').execute()
# comments_result = []
# comments_result.append(comments)
# comments_result

In [ ]:
#commentThreads  information about a YouTube comment thread, which comprises a top-level comment and replies, if any exist, to that comment. 
# filters : allThreadsRelatedToChannelId , channelId, id, videoId   -- channelId로 했을때는 정보 안나옴  

#videoid : sCcSBEw2DmY
#videoid로 할때 : 최상단 고정 댓글과 좋아요 많이 받은 댓글들을 긁어오고(toplevelcomments), 그 댓글의 대댓글까지 긁어옴(replies).
#order : relevance 하면 좋아요 순으로 가져옴

#channelid : UCbFzvzDu17eDZ3RIeaLRswQ
#all~~channelId로 할때 : 각 영상의 최신 댓글을 긁어 오는듯? 아니면 채널 전체의 최신 댓글. -- replies 리소스 안넘어옴 
#순서 못바꾸는듯. 


# commentThreads = youtube.commentThreads().list(part='id, replies, snippet', allThreadsRelatedToChannelId='UCbFzvzDu17eDZ3RIeaLRswQ', fields='pageInfo, nextPageToken, items(id, snippet(videoId, topLevelComment(snippet( textOriginal, authorDisplayName, likeCount)), totalReplyCount), replies)', order='relevance').execute()
commentThreads = youtube.commentThreads().list(part='id, replies, snippet', videoId='sCcSBEw2DmY', fields='pageInfo, nextPageToken, items(id, snippet(videoId, topLevelComment(snippet( textOriginal, authorDisplayName, likeCount)), totalReplyCount), replies(comments(snippet(textOriginal,authorDisplayName, likeCount))))', order='relevance').execute()
commentThreads_result = []
commentThreads_result.append(commentThreads)
commentThreads_result

In [ ]:
# guideCategories   YouTube 알고리즘이 채널의 콘텐츠 또는 인기도와 같은 기타 지표를 바탕으로 지정한 카테고리를 확인합니다
#이딴 메소드 없음 


# guideCategories = youtube.guideCategories().list(part='id, snippet', regionCode='KR').execute()
# guideCategories

In [ ]:
# playlistitems, playlists 지금은 쓸데없는듯

In [ ]:
# search API 요청에 지정된 검색 매개변수와 일치하는 YouTube 동영상, 채널 또는 재생목록의 정보가 포함되어 있습니다.

#정말로 검색을 하는 것. 유튜브에서 검색하는거라고 생각하면 됨.

#relatedToVideoId : 해당 영상과 관련된 영상들을 찾아준다.(반드시 영상 단위로 밖에 안됨. type='video' 필수)  연관 유튜버 찾는데 활용 가능할듯 

#선택적 매개변수 : channelId(해당 채널에서 만든 영상만 리턴), order(date, rating, relevance, title, videoCount, viewCount), q(검색어 입력), type(channel, playlist, video), videoCategoryId(카테고리에 맞는 것만 리턴), regionCode(국가 설정 KR)

# search = youtube.search().list(part='id, snippet', relatedToVideoId='sCcSBEw2DmY', type='video').execute()
# search = youtube.search().list(part='id, snippet', q='힙합', type='channel', order='viewCount', regionCode='KR' ).execute()
search = youtube.search().list(part=' snippet', channelId='UCbFzvzDu17eDZ3RIeaLRswQ', type='video', order='viewCount').execute()
search

In [ ]:
#subscription 사용자의 구독정보 

#되는 채널이 있고 안되는 채널이 잇음 자신의 구독 정보 오픈 여부에 따라 다른듯
#대부분 안됨.

subscription = youtube.subscriptions().list(part='id, snippet, contentDetails', channelId='UCUbOogiD-4PKDqaJfSOTC0g').execute()

subscription

In [ ]:
# Thumbnails  영상 썸네일 가져오기 
# videoAbuseReportReasons 쓸데없음

# videoCategories  연결 가능한 카테고리 목록을 가져옴 (어떤 영상의 카테고리를 알려주는게 아님 . 가능한 카테고리 목록을 반환하는 거임)

videoCategories = youtube.videoCategories().list(part='id, snippet', regionCode='KR').execute()

category_list = []
for i in range(1,31):
    category_list.append(videoCategories['items'][i]['snippet']['title'])
    print(videoCategories['items'][i]['snippet']['title'])

In [ ]:
# videos  동영상에 대한 정보

# 동영상 소유자만 되는 메소드 : fileDetails, processingDetails, suggestions

#part : id, snippet, contentDetails, fileDetails, liveStreamingDetails, player, processingDetails, recordingDetails, statistics, status, suggestions, topicDetails
#필터 : chart(mostPopular)-지정된 콘텐츠 지역 및 동영상 카테고리의 인기 동영상 반환(regionCode, videoCategoryId) , id-영상id로 검색


videos = youtube.videos().list(part='id, snippet,  recordingDetails, statistics,  topicDetails', id='sCcSBEw2DmY', fields='items(snippet(title, description, tags, categoryId),id,  recordingDetails, statistics,  topicDetails )').execute()
videos